In [1]:
import json
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from tensorflow import keras

import autokeras as ak

from bokeh.plotting import figure, show
from bokeh.io import output_notebook


output_notebook()

Loading BokehJS ...

Lets start by loading our data from our JSON data file. Then we will see how many samples of what length and for which commands we have to see which length to choose.

In [2]:
datafile = './20200223_data_200ms_right_up_left_down.dat'
with open(datafile,'r') as f:
    data = json.loads(f.read())
for sample in data:
    sample['vectors']=np.array(sample['vectors'])

cnt_list = []
for vector_len in range(10):
    cnt = len([0 for sample in data if len(sample['vectors']) == vector_len])
    cnt_list.append(cnt)
    print("samples of len {}: {}".format(vector_len,cnt))

cmd_set = set([e['command'] for e in data])
n_cmds = len(cmd_set)
cnt_max = np.max(cnt_list)
cnt_argmax = np.argmax(cnt_list)


for cmd in range(n_cmds):
    n_samps = len([0 for e in data if e['command'] == cmd and len(e['vectors']) == cnt_argmax])
    print("len: {}, command: {}, samples: {}".format(cnt_argmax,cmd,n_samps))

samples of len 0: 0
samples of len 1: 0
samples of len 2: 12
samples of len 3: 314
samples of len 4: 475
samples of len 5: 60
samples of len 6: 0
samples of len 7: 0
samples of len 8: 0
samples of len 9: 0
len: 4, command: 0, samples: 97
len: 4, command: 1, samples: 104
len: 4, command: 2, samples: 142
len: 4, command: 3, samples: 132


So it looks like the mode of our length of samples is 4 with 475 samples, and for this length it looks like we have pretty good distribution amongst the various commands. 

Now we don't always want to be infering one of our commands, so we will also have noise commands which won't result in any action and will hopefully be inferred when the wand is sitting still for instance. To create these "noise" samples I just did long records without making any specific gestures and then below we randomly sample from this long file.

In [3]:
noisefile = './20200224_noise.dat'
with open(noisefile,'r') as f:
    noise = json.loads(f.read())
    
noise_array = []
    
for sample in noise:
    noise_array += sample['vectors']

noise_array = np.array(noise_array)

noise_cmd = max(cmd_set)+1
for _ in range(int(cnt_max/n_cmds)):
    idx = np.random.randint(0,noise_array.shape[0]-cnt_argmax)
    data.append({'command':noise_cmd,
                 'vectors':noise_array[idx:idx+cnt_argmax,:]})
    
cmd_set = set([e['command'] for e in data])
n_cmds = len(cmd_set)

for cmd in range(n_cmds):
    n_samps = len([0 for e in data if e['command'] == cmd and len(e['vectors']) == cnt_argmax])
    print("len: {}, command: {}, samples: {}".format(cnt_argmax,cmd,n_samps))

len: 4, command: 0, samples: 97
len: 4, command: 1, samples: 104
len: 4, command: 2, samples: 142
len: 4, command: 3, samples: 132
len: 4, command: 4, samples: 118


Now we can create our input samples X and our output classes Y, and then do a train-test split.

In [4]:
n_features = data[0]['vectors'].shape[-1]
n_lookback = cnt_argmax
n_commands = n_cmds

In [5]:
X = np.array([s['vectors'].flatten() for s in data if len(s['vectors']) == cnt_argmax])
y = np.array([s['command'] for s in data if len(s['vectors']) == cnt_argmax])
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=.5)

## Super simple 1 dense layer model

Now lets make a super simple model that consists of just two dense layers.

In [6]:
inputs = keras.layers.Input(shape=(cnt_argmax*n_features,))
x = inputs
# x = keras.layers.Dense(128,activation='linear')(x)
outputs = keras.layers.Dense(n_cmds,activation='linear')(x)

model = keras.Model(inputs,outputs)
model.summary()

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24)]              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 125       
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________


Now we train our model with an early stopping callback.

In [7]:
es = keras.callbacks.EarlyStopping(monitor='accuracy',min_delta=0.1,patience=5)

model.fit(X_train,y_train,validation_split=0.1,epochs=1000,verbose=False)
loss, acc = model.evaluate(X_test,y_test)
print("the test accuracy is: {}".format(acc))

297/297 [==============================] - ETA: 0s - loss: 0.9690 - accuracy: 0.87 - 0s 36us/sample - loss: 0.7109 - accuracy: 0.8788
the test accuracy is: 0.8787878751754761


Now lets print out the confusion matrix to see how our model did.

In [8]:
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred,-1)
confusion_matrix(y_pred,y_test)

array([[37,  6,  0,  0,  3],
       [ 1, 50,  1,  0,  5],
       [ 1,  1, 71,  1,  0],
       [ 0,  1,  5, 51,  1],
       [ 7,  1,  1,  1, 52]])

That confusion matrix looks pretty good, it seems like the command that gets confused the most is command 3 gets confused with command 2 4 times. 

Now we have a very simple model from which we can grab the weights and format them into constants for the arduino inference code.

## Auto Keras Model

Now, for fun lets try generating a model with Auto Keras. Before we begin, I expect this model to have better accuracy, however I also expect it to be more complex (more layers, more complex activation functions etc.)

In [9]:
clf = ak.StructuredDataClassifier(max_trials=10)

In [10]:
clf.fit(X_train,y_train,verbose=False)
clf.evaluate(X_test,y_test)

INFO:tensorflow:Oracle triggered exit
10/10 [==============================] - ETA: 1s - loss: 1.1254 - accuracy: 0.87 - 0s 16ms/step - loss: 0.4961 - accuracy: 0.9327


[0.4960911226691678, 0.9326599]

In [11]:
auto_model = clf.export_model()
auto_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24)]              0         
_________________________________________________________________
categorical_encoding (Catego (None, 24)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               3200      
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
re_lu (ReLU)                 (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
_________________________________________________________________
classification_head_1 (Softm (None, 5)                 0     

In [12]:
y_pred_auto = clf.predict(X_test)
confusion_matrix(y_pred_auto,y_test)

array([[44,  5,  2,  0,  1],
       [ 1, 53,  0,  0,  0],
       [ 0,  0, 68,  0,  0],
       [ 0,  1,  8, 52,  0],
       [ 1,  0,  0,  1, 60]])

In [13]:
auto_loss, auto_acc = clf.evaluate(X_test,y_test)
print("the test accuracy of the auto model is: {}".format(auto_acc))

10/10 [==============================] - ETA: 1s - loss: 1.1254 - accuracy: 0.87 - 0s 15ms/step - loss: 0.4961 - accuracy: 0.9327
the test accuracy of the auto model is: 0.932659924030304


So we can see that we this model does have better accuracy (.94 vs .88) however it is much more complex in terms of number of layers and type of layers. Because our model will be running on an embedded microcontroller, and we have to hand code the model, we will stick with the 1-layer model.

## Model export

Ok, since I am not aware of an arduino api for tensorflow (I know there is a C/C++ one, however getting this to work on an ATMega chip is probably more trouble than its worth), and since this is a super simple model, we can just hand code it in the Arduino API.

First thing first lets make sure we understand the matrix multiply and bias matrix dimensions.

In [14]:
weights = model.layers[1].get_weights()[0]
biases = model.layers[1].get_weights()[1]

In [15]:
Y_pred_simple = X_test.dot(weights) + biases
y_pred_simple = np.argmax(Y_pred_simple,axis=1)
confusion_matrix(y_pred_simple,y_pred)

array([[46,  0,  0,  0,  0],
       [ 0, 57,  0,  0,  0],
       [ 0,  0, 74,  0,  0],
       [ 0,  0,  0, 58,  0],
       [ 0,  0,  0,  0, 62]])

So we can see that because this was a simple 1-layer model, we can simply matrix multiply by the weight matrix, add the biases, and then take the argmax for inference. 

Now lets print out these arrays in a format that we can easily copy over to our arduino code.

In [16]:
weights_str = "{"
for i, row in enumerate(weights):
    weights_str += "{"
    for j, col in enumerate(row):
        weights_str += str(col)
        if j < len(row)-1:
            weights_str += ","
    weights_str += "}"
    if i < len(weights)-1:
        weights_str += ",\n"
weights_str += "};"

Now this string can be coppied into an Arduino code, or if you want you can write some fancy python string manipulation to generate the source code file directly from python, that would be cool. In fact lets do that.

In [17]:
biases_str = "{"
for i, e in enumerate(biases):
    biases_str += str(e)
    if i < len(biases)-1:
        biases_str += ","
biases_str += "};"
print(biases_str)

{-1.551105,-0.90070885,-0.8249986,-1.7614192,3.1447637};


In [18]:
input_str = "{" + "0.0,"*(n_lookback*n_features-1)+"0.0};"
print(input_str)

{0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0};


In [19]:
output_str = "{" + "0.0,"*(n_commands-1)+"0.0};"
print(output_str)

{0.0,0.0,0.0,0.0,0.0};


In [20]:
str_dict = {'n_lookback_str':str(n_lookback),
            'n_features_str':str(n_features),
            'n_commands_str':str(n_commands),
            'input_str':input_str,
            'output_str':output_str,
            'weights_str':weights_str,
            'biases_str':biases_str}

Sweeto burrito, now we have a dictionary of strings, and I have created a template file where the keys in this dictionary are where the values should be placed in the template file. So if we change the number of features, the number of lookback steps, or relearn the model, we can run this notebook and regenerate the arduino (C) code.

In [21]:
input_file = './infer_command_NXP-9-DOF/infer_command_NXP-9-DOF.ino.template'
output_file = './infer_command_NXP-9-DOF/infer_command_NXP-9-DOF.ino'

with open(input_file,'r') as f:
    input_f_str = f.read()

for key, val in str_dict.items():
    input_f_str = input_f_str.replace(key,val)

with open(output_file,'w') as f:
    f.write(input_f_str)